In [155]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt


logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73")]

In [156]:
list(edb.get_timeseries_db().find({"metadata.key": "config/sensor_config", "data.battery_status": {"$exists": True}}).sort("metadata.write_ts").limit(10))

[]

In [158]:
#type: list of emission.storage.timeseries.builtin_timeseries.BuiltinTimeSeries
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

#time range specification
dt1 = pydt.datetime(2016, 6, 24, 22, 0) #datetime.datetime(2016, 6, 24, 22, 0)
dt2 = pydt.datetime(2016, 6, 26, 23, 55)

ts1 = int(dt1.strftime("%s"))
ts2 = int(dt2.strftime("%s"))

time_range = estt.TimeQuery("metadata.write_ts", ts1, ts2)

#BuiltinTimeSeries.get_data_df(key, time_query = None, geo_query = None, extra_query_list=None) 
#type: list of DataFrame objects (regarding state info)
iphone_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), iphone_ts)
android_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), android_ts)

iphone_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), iphone_ts)
android_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), android_ts)

In [165]:
_iphone_config_df_list = map(lambda t:t.get_data_df("config/sensor_config"), iphone_ts)
_android_config_df_list = map(lambda t:t.get_data_df("config/sensor_config"), android_ts)

ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)

i_config_df_list=[]
for df in _iphone_config_df_list:
    df['local_dt'] = df['metadata_write_ts'].apply(ts_to_dt)
    df['ts'] = df['metadata_write_ts']
    i_config_df_list.append(df)

a_config_df_list=[]
for df in _android_config_df_list:
    df['local_dt'] = df['metadata_write_ts'].apply(ts_to_dt)
    df['ts'] = df['metadata_write_ts'] #<class 'pandas.core.series.Series'>
    a_config_df_list.append(df)

In [144]:
#query info from a specified time range
iphone_state_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _iphone_state_df_list)
android_state_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _android_state_df_list)

iphone_battery_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _iphone_battery_df_list)
android_battery_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _android_battery_df_list)

iphone_config_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], i_config_df_list)
android_config_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], a_config_df_list)

In [145]:
def config_changed(df):
    changed_lst = ['Null']
    temp = df.copy()
    temp = temp.drop(['_id', 'metadata_write_ts', 'local_dt', 'ts'], 1)
    #print temp
    #temp.reset_index(inplace=True)
    for i in range(1, len(temp)):
        #print(temp.iloc[i]==temp.iloc[i-1])
        if temp.iloc[i].equals(temp.iloc[i-1]):
            changed_lst.append('0')
        else:
            changed_lst.append('1')
    return changed_lst

In [146]:
iphone_config_change_df_list=[]
for index, df in enumerate(iphone_config_df_list):
    config_series = config_changed(df)
    df.loc[:, 'config_changed'] = pd.Series(config_series, index=df.index)
    iphone_config_change_df_list.append(df)
    
android_config_change_df_list=[]
for index, df in enumerate(android_config_df_list):
    config_series = config_changed(df)
    df.loc[:, 'config_changed'] = pd.Series(config_series, index=df.index)
    android_config_change_df_list.append(df)

In [147]:
ios1 = iphone_config_change_df_list[0]
ios1
android1 = android_config_change_df_list[2]
android1

,_id,accuracy,accuracy_threshold,android_geofence_responsiveness,filter_distance,filter_time,geofence_radius,ios_use_remote_push_for_sync,ios_use_visit_notifications_for_detection,is_duty_cycling,metadata_write_ts,simulate_user_interaction,trip_end_stationary_mins,local_dt,ts,config_changed
1,576e3a0b88f6636990e50f5a,102,200,5000,-1,10,100,False,False,True,1.466833e+09,False,5,2016-06-24 22:29:51.618,1.466833e+09,Null
2,576e3a0b88f6636990e50f5b,102,200,5000,-1,10,100,False,False,False,1.466833e+09,False,5,2016-06-24 22:29:56.262,1.466833e+09,1
3,576e3a0b88f6636990e50f5c,100,200,5000,-1,10,100,False,False,False,1.466833e+09,False,5,2016-06-24 22:30:01.278,1.466833e+09,1
4,576e3a0f88f6636990e50f5d,100,200,5000,-1,10,100,False,False,False,1.466834e+09,False,5,2016-06-24 22:51:36.700,1.466834e+09,0


In [148]:
#checking iOS data...
n = 2
ios_s = iphone_state_df_list[n]
ios_b = iphone_battery_df_list[n]
ios_c = iphone_config_change_df_list[n]

ios_s_new = ios_s[(ios_s.transition!=7) & (ios_s.transition!=16)]
ios_combined = pd.merge(ios_s_new, ios_c, on='ts', how='outer')
ios_combined.sort_values('ts', ascending=True, inplace=True)
ios_combined['dt']=ios_combined['ts'].apply(ts_to_dt)

#print len(ios_combined)
ios_combined.fillna('-')[['fmt_time','local_dt_y','curr_state', 'transition', 'config_changed', 'ios_use_visit_notifications_for_detection']]
#ios_combined

,fmt_time,local_dt_y,curr_state,transition,config_changed,ios_use_visit_notifications_for_detection
0,2016-06-24T22:11:01.277140-07:00,-,2,3,-,-
1,2016-06-24T22:11:01.379465-07:00,-,2,11,-,-
16,-,2016-06-24 22:41:02.399021,-,-,Null,False
2,2016-06-24T22:41:02.438633-07:00,-,3,3,-,-
3,2016-06-24T22:41:02.529346-07:00,-,3,11,-,-
4,2016-06-24T22:41:02.601251-07:00,-,3,17,-,-
5,2016-06-24T22:41:02.676155-07:00,-,0,0,-,-
6,2016-06-24T22:41:02.776184-07:00,-,2,6,-,-
17,-,2016-06-24 22:46:43.466136,-,-,1,False
7,2016-06-24T22:46:43.504092-07:00,-,2,3,-,-


In [151]:
#checking Android data...
n = 2

a_s = android_state_df_list[n]
a_b = android_battery_df_list[n]
a_c = android_config_change_df_list[n]

a_combined = pd.merge(a_s, a_c, on='ts', how='outer')
a_combined.sort_values('ts', ascending=True, inplace=True)
a_combined['dt']=a_combined['ts'].apply(ts_to_dt)

print len(a_combined)
a_combined.fillna('-')[['fmt_time','local_dt_y', 'curr_state', 'transition', 'config_changed']]

a_all = pd.merge(a_combined, a_b, on='ts', how='outer')
a_all.sort_values('ts', ascending=True, inplace=True)
a_all.rename(columns = {'fmt_time_x':'state_time', 'local_dt_y':'config_time', 'fmt_time_y':'battery_time'}, inplace = True)
a_all.fillna('-')[['state_time','config_time', 'battery_time','curr_state', 'transition', 'config_changed', 'battery_level_pct', 'battery_status']]
#a_bs_combined = pd.merge(a_s, a_b, on='ts', how='outer')
#a_bs_combined = a_bs_combined[['ts','fmt_time_x', 'curr_state', 'transition', 'fmt_time_y', 'battery_level_pct', 'battery_status']]

#a_bs_combined.sort_values('ts', ascending=True, inplace=True)
#a_bs_combined

43


,state_time,config_time,battery_time,curr_state,transition,config_changed,battery_level_pct,battery_status
43,-,-,2016-06-24T22:00:09.749000-07:00,-,-,-,96,1
44,-,-,2016-06-24T22:01:08.634000-07:00,-,-,-,96,1
0,2016-06-24T22:02:53.482000-07:00,-,-,1,3,-,-,-
45,-,-,2016-06-24T22:02:53.525000-07:00,-,-,-,96,1
46,-,-,2016-06-24T22:03:07.179000-07:00,-,-,-,96,1
47,-,-,2016-06-24T22:04:08.064000-07:00,-,-,-,96,1
48,-,-,2016-06-24T22:05:08.884000-07:00,-,-,-,96,1
49,-,-,2016-06-24T22:07:08.841000-07:00,-,-,-,96,1
50,-,-,2016-06-24T22:09:08.790000-07:00,-,-,-,96,1
51,-,-,2016-06-24T22:10:09.039000-07:00,-,-,-,96,1
